In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.8.2'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import matplotlib.pyplot as plt
from google.colab import files

%matplotlib inline

In [5]:
!unzip "/content/drive/MyDrive/yolo3/darknet.zip" -d "/content"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_03_988.txt  
  inflating: /content/__MACOSX/darknet/data/images/val/Knife/._H_8211.91-0000_03_988.txt  
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_13_988.txt  
  inflating: /content/__MACOSX/darknet/data/images/val/Knife/._H_8211.91-0000_13_988.txt  
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_21_376.txt  
  inflating: /content/__MACOSX/darknet/data/images/val/Knife/._H_8211.91-0000_21_376.txt  
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_21_410.txt  
  inflating: /content/__MACOSX/darknet/data/images/val/Knife/._H_8211.91-0000_21_410.txt  
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_10_593.jpg  
  inflating: /content/__MACOSX/darknet/data/images/val/Knife/._H_8211.91-0000_10_593.jpg  
  inflating: /content/darknet/data/images/val/Knife/H_8211.91-0000_06_629.txt  
  inflating: /content/__MACOS

In [6]:
pwd

'/content'

In [7]:
import shutil
import os

source_path='/content/darknet/data/images/train'

train_list=[]
for (path,directory,files) in os.walk(source_path):
  for file in files:
    if(file == '.DS_store'): continue
    train = path + '/' + file
    train_list.append(train)

print(len(train_list))
with open('/content/darknet/data/list/train.txt','w') as f:
  for train in train_list:
    f.write(train+'\n')



150677


In [8]:
import shutil
import os

source_path='/content/darknet/data/images/test'

test_list=[]
for (path,directory,files) in os.walk(source_path):
  for file in files:
    if(file == '.DS_store'): continue
    test = path + '/' + file
    test_list.append(test)

print(len(test_list))
with open('/content/darknet/data/list/test.txt','w') as f:
  for train in test_list:
    f.write(test+'\n')



31349


In [9]:
import shutil
import os

source_path='/content/darknet/data/images/val'

val_list=[]
for (path,directory,files) in os.walk(source_path):
  for file in files:
    if(file == '.DS_store'): continue
    val = path + '/' + file
    val_list.append(val)

print(len(val_list))
with open('/content/darknet/data/list/val.txt','w') as f:
  for val in val_list:
    f.write(val+'\n')



37702


In [10]:
# data 파일 수정 = .txt 지우기 ^^
# cfg 파일 수정
'''
1. 데이터 총합 = 219729
- classes = 38
- 416 x 416
- batch 64 , sub =16
- max_batches 38*1000 = 38000
- 러닝메이트
- 38000**0.8=30400 , 38000*0*.9=34200
- filter 129
'''
# 또 에러 나면 배치 사이즈 64 ~>32 로 줄이기

'\n1. 데이터 총합 = 219729\n- classes = 38\n- 416 x 416\n- batch 64 , sub =16\n- max_batches 38*1000 = 38000\n- 러닝메이트\n- 38000**0.8=30400 , 38000*0*.9=34200\n- filter 129\n'

In [11]:
# 우리가 사용할 YOLOv4의 구조를 살펴보자.
%cat /content/darknet/cfg/yolov3.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 38000
policy=steps
steps=30400,34200
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutional]
ba

In [12]:
cd /content/darknet

/content/darknet


In [ ]:
# yolov3 pre-trained weights 다운로드
!wget wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-30 04:24:18--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2022-08-30 04:24:18--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.6MB/s    in 8.3s    

2022-08-30 04:24:28 (18.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]

FINISHED --2022-08-30 04:24:28--
Total wall clock time: 9.2s
Downloaded: 1 files, 155M in 8.3s (18.6 MB/s)


In [13]:
!ln -s /content/drive/MyDrive /content/darknet/mydrive
!ls /content/darknet/mydrive

 정리			      'Colab Notebooks'   object_Detect   yolo3
 211101_object_sbppt.gslides   github_anna	  project	  yolo4


In [21]:
# GPU, LIBSO을 false로 설정
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=1/GPU=0/g' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/g' Makefile
!apt update
!apt-get install libopencv-dev
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile
!sed -i 's/LIBSO=1/LIBSO=0/' Makefile


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

In [14]:
# GPU, OPENCV, LIBSO을 true로 설정
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt update
!apt-get install libopencv-dev
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

In [22]:
# 위의 변경된 옵션으로 makefile을 컴파일하면 darknet.py 사용이 가능해진다.
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1150:10: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the 

In [23]:
cd /content/darknet

/content/darknet


In [24]:
ls

3rdparty/               darknet_video.py        net_cam_v4.sh*
backup/                 data/                   obj/
build/                  image_yolov3.sh*        README.md
build.ps1*              image_yolov4.sh*        results/
cfg/                    include/                scripts/
cmake/                  json_mjpeg_streams.sh*  src/
CMakeLists.txt          libdarknet.so*          uselib*
darknet*                LICENSE                 vcpkg.json
DarknetConfig.cmake.in  Makefile                video_yolov3.sh*
darknet_images.py       mydrive@                video_yolov4.sh*
darknet.py              net_cam_v3.sh*


In [25]:
#/content/drive/MyDrive/yolo3/1/yolov3_last.weights
#2. 학습
!./darknet detector train data/xrayDatas.data cfg/yolov3.cfg /content/drive/MyDrive/yolo3/1/yolov3_last.weights -dont_show -map

 GPU isn't used 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177

In [ ]:
# 1. 학습
# custom dataset으로 YOLO 학습 (아래 코드는 model.fit 코드라고 이해)
# data 필요, model 구조 와 세부설정 값, model weight
!./darknet detector train data/xrayDatas.data cfg/yolov3.cfg darknet53.conv.74 -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv   

In [ ]:
!unzip "/content/drive/MyDrive/yolo4/test.zip" -d "/content/darknet/data/test"

Archive:  /content/drive/MyDrive/yolo4/test.zip
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_630.jpg  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_630.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_600.jpg  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_634.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_648.jpg  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_616.jpg  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_623.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_634.jpg  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_636.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_625.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_656.txt  
  inflating: /content/darknet/data/test/images/H_8201.40-0000_08_600.txt  
  inflating: /content/darknet/data/test/images/H_820

In [ ]:
# 평가해보기
! cd /content/darknet
!./darknet detector test data/xrayDatas.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/mydrive/yolo4/2/yolov4-custom_best.weights /content/darknet/data/images/S_8544.42-2090_08_241.jpg

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    480 x 480 x   3 ->  480 x 480 x  32 0.398 BF
   1 conv     64       3 x 3/ 2    480 x 480 x  32 ->  240 x 240 x  64 2.123 BF
   2 conv     64       1 x 1/ 1    240 x 240 x  64 ->  240 x 240 x  64 0.472 BF
   3 route  1 		                           ->  240 x 240 x  64 
   4 conv     64       1 x 1/ 1    240 x 240 x  64 ->  240 x 240 x  64 0.472 BF
   5 conv     32       1 x 1/ 1    240 x 240 x  64 ->  240 x 240 x  32 0.236 BF
   6 conv     64       3 x 3/ 1    240 x 240 x  32 ->  240 x 240 x  64 2.123 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 240 x 240 x  64 0.004 BF
   8 conv  

In [ ]:
! sudo apt-get install x11-xserver-utils
! xhost + # everyone can access x11


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  nickle cairo-5c xorg-docs-core
The following NEW packages will be installed:
  x11-xserver-utils
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 159 kB of archives.
After this operation, 517 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-xserver-utils amd64 7.7+7build1 [159 kB]
Fetched 159 kB in 0s (327 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a contr

In [ ]:
! xhost + # everyone can access x11


xhost:  unable to open display ""
